In [1]:
import geopandas as gpd
import pandas as pd
import os
from glob import glob
from sklearn.linear_model import LinearRegression
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, root_mean_squared_error

In [ ]:
cells = gpd.read_file('global-inputs/HR6-change-cells-aoi.gpkg')

cell_ids = cells.hex_id.tolist()

In [ ]:
shoreline_files = pd.Series(sorted(glob("data/HR6/*/shoreline_timeseries_tidal_correction.csv")))

0      data/HR6/86ba24017ffffff/shoreline_timeseries_...
1      data/HR6/86ba24037ffffff/shoreline_timeseries_...
2      data/HR6/86ba24067ffffff/shoreline_timeseries_...
3      data/HR6/86ba2414fffffff/shoreline_timeseries_...
4      data/HR6/86ba2415fffffff/shoreline_timeseries_...
                             ...                        
765    data/HR6/86dab3717ffffff/shoreline_timeseries_...
766    data/HR6/86dab371fffffff/shoreline_timeseries_...
767    data/HR6/86dab3737ffffff/shoreline_timeseries_...
768    data/HR6/86dab3757ffffff/shoreline_timeseries_...
769    data/HR6/86dab375fffffff/shoreline_timeseries_...
Length: 770, dtype: object

In [9]:
def calc_trends(file):
    df  = pd.read_csv(file)
    try:
        df.date = pd.to_datetime(df.date)
    except:
        print(file)
    df.index = (df.date - df.date.min()).dt.days / 365.25
    df.drop(columns=['date', 'Unnamed: 0', 'corrected_IW_shoreline_position', 'EOV_shoreline_chg'], inplace=True, errors='ignore')
    trends = {}
    for col in df.columns:
        sub_df = df[col].dropna()
        
        if not len(sub_df):
            continue
        x = sub_df.index.to_numpy().reshape(-1,1)
        y = sub_df
        linear_trend = LinearRegression().fit(x,y)
        pred = linear_trend.predict(x)

        trends['index'] = file.split('/')[2]
        if col == 'normalised_IW_shoreline_position':
            trends["IW shoreline trend"] = linear_trend.coef_[0]
            trends["IW shoreline intercept"] = linear_trend.intercept_
            trends["IW shoreline n_points"] = len(df[col])
            trends["IW shoreline n_points_nonan"] = len(sub_df)
            trends["IW shoreline r2_score"] = r2_score(y, pred)
            trends["IW shoreline mae"] = mean_absolute_error(y, pred)
            trends["IW shoreline mse"] = mean_squared_error(y, pred)
            trends["IW shoreline rmse"] = root_mean_squared_error(y, pred)
        else:
            trends["EOV shoreline trend"] = linear_trend.coef_[0]
            trends["EOV shoreline intercept"] = linear_trend.intercept_
            trends["EOV shoreline n_points"] = len(df[col])
            trends["EOV shoreline n_points_nonan"] = len(sub_df)
            trends["EOV shoreline r2_score"] = r2_score(y, pred)
            trends["EOV shoreline mae"] = mean_absolute_error(y, pred)
            trends["EOV shoreline mse"] = mean_squared_error(y, pred)
            trends["EOV shoreline rmse"] = root_mean_squared_error(y, pred)

    return pd.DataFrame([trends])
    

In [10]:
trends = pd.concat([calc_trends(f) for f in shoreline_files])

trends

/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal-monitoring/lib/python3.13/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
/home/ubuntu/miniforge3/envs/coastal

,index,IW shoreline trend,IW shoreline intercept,IW shoreline n_points,IW shoreline n_points_nonan,IW shoreline r2_score,IW shoreline mae,IW shoreline mse,IW shoreline rmse,EOV shoreline trend,EOV shoreline intercept,EOV shoreline n_points,EOV shoreline n_points_nonan,EOV shoreline r2_score,EOV shoreline mae,EOV shoreline mse,EOV shoreline rmse
0,86ba24017ffffff,0.754761,2.777960,157.0,157.0,0.058314,11.894362,190.073993,13.786733,-0.771420,4.229877,157.0,157.0,0.015716,23.809367,770.069567,27.750127
0,86ba24037ffffff,1.072725,-6.667474,159.0,159.0,0.368984,6.853272,65.646662,8.102263,1.599328,-20.920154,159.0,159.0,0.420149,5.748990,117.758257,10.851648
0,86ba24067ffffff,31.976518,-728.632013,211.0,211.0,0.071408,218.175154,338651.246857,581.937494,0.613516,-17.350338,211.0,211.0,0.010597,22.202602,895.038904,29.917201
0,86ba2414fffffff,0.370716,4.910991,174.0,174.0,0.019038,6.629248,105.910401,10.291278,0.646561,-15.527652,174.0,174.0,0.027285,8.235120,222.899396,14.929816
0,86ba2415fffffff,0.293307,2.322676,286.0,286.0,0.003081,15.147171,758.722732,27.544922,-0.442925,-8.486781,286.0,286.0,0.003635,27.990655,1465.947505,38.287694
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,86dab3717ffffff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.018919,-14.912824,205.0,205.0,0.000251,6.875102,68.685599,8.287678
0,86dab371fffffff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.178338,-19.369095,180.0,180.0,0.002507,18.131009,547.891205,23.407076
0,86dab3737ffffff,1.953625,-13.843882,162.0,162.0,0.632647,7.979527,126.080045,11.228537,0.072761,-1.201097,162.0,162.0,0.051363,0.938234,5.562730,2.358544
0,86dab3757ffffff,-0.678629,22.505016,191.0,191.0,0.073018,15.478607,335.515969,18.317095,0.326063,9.973266,191.0,191.0,0.069691,7.525995,81.445040,9.024691


In [11]:
trends_file_path = 'shoreline_trends_by_cell.geojson'
# if os.path.exists(trends_file_path):
#     cell_trends = gpd.read_file('shoreline_trends_by_cell.geojson')
#     cell_trends.to_crs("EPSG:4326", inplace=True)
#     cell_trends.update(trends)
#     cell_trends.to_file(trends_file_path)
# else:
cell_trends = cells.merge(trends, on='index')
cell_trends.to_crs("EPSG:4326", inplace=True)
cell_trends.to_file(trends_file_path)
    

cell_trends

,index,hex_id,parent_id,sediment_area_ha,geometry,IW shoreline trend,IW shoreline intercept,IW shoreline n_points,IW shoreline n_points_nonan,IW shoreline r2_score,...,IW shoreline mse,IW shoreline rmse,EOV shoreline trend,EOV shoreline intercept,EOV shoreline n_points,EOV shoreline n_points_nonan,EOV shoreline r2_score,EOV shoreline mae,EOV shoreline mse,EOV shoreline rmse
0,86bb5e18fffffff,86bb5e18fffffff,85bb5e1bfffffff,16.76,"POLYGON ((173.94469 -35.12657, 173.94025 -35.0...",0.286229,-4.320310,129.0,129.0,0.026929,...,107.405463,10.363661,1.256841,-22.462887,129.0,129.0,0.064800,21.657042,827.099113,28.759331
1,86bb5e19fffffff,86bb5e19fffffff,85bb5e1bfffffff,20.00,"POLYGON ((174.01618 -35.12027, 174.01171 -35.0...",0.474741,2.817206,77.0,77.0,0.073889,...,59.034851,7.683414,0.526519,-5.437018,77.0,77.0,0.049005,9.889406,112.430338,10.603317
2,86bb29447ffffff,86bb29447ffffff,85bb2947fffffff,5.76,"POLYGON ((174.77825 -41.21253, 174.77308 -41.1...",4.020678,16.197371,199.0,199.0,0.250750,...,1922.296508,43.844002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,86bb58027ffffff,86bb58027ffffff,85bb5803fffffff,97.20,"POLYGON ((172.71095 -34.47891, 172.70704 -34.4...",0.692400,5.344274,22.0,22.0,0.052207,...,97.335193,9.865860,27.477661,-88.255125,22.0,22.0,0.052533,328.470724,152285.986766,390.238372
4,86bb72b4fffffff,86bb72b4fffffff,85bb72b7fffffff,14.08,"POLYGON ((173.87492 -39.18614, 173.87027 -39.1...",-0.363870,26.446637,143.0,143.0,0.004920,...,636.475398,25.228464,1.975454,-61.039399,143.0,143.0,0.058542,33.910147,1491.550908,38.620602
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,86da91d8fffffff,86da91d8fffffff,85da91dbfffffff,5.36,"POLYGON ((171.05489 -42.73298, 171.05131 -42.6...",-0.744172,11.424185,222.0,222.0,0.054063,...,432.702328,20.801498,0.093831,-0.356867,222.0,222.0,0.302556,0.804548,0.906301,0.951998
740,86bb299afffffff,86bb299afffffff,85bb299bfffffff,11.68,"POLYGON ((174.2819 -41.84778, 174.27691 -41.81...",2.592510,-168.383562,309.0,309.0,0.231783,...,796.291844,28.218644,-4.952195,-100.780524,309.0,309.0,0.211410,44.162292,3270.019879,57.184088
741,86da92bb7ffffff,86da92bb7ffffff,85da92bbfffffff,13.40,"POLYGON ((171.93734 -41.54563, 171.93341 -41.5...",1.172277,-4.493498,187.0,187.0,0.041993,...,701.826681,26.492012,0.590607,0.705171,187.0,187.0,0.014208,10.819057,541.777267,23.276109
742,86dab3717ffffff,86dab3717ffffff,85dab373fffffff,35.00,"POLYGON ((172.69314 -43.80127, 172.68874 -43.7...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-0.018919,-14.912824,205.0,205.0,0.000251,6.875102,68.685599,8.287678
